# Intrinsic Analysis

In [1]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import yaml
import matplotlib.pyplot as plt
import contextily as cx
from src import graph_functions as gf
from src import simplification_functions as sf
from src import evaluation_functions as ef

In [56]:
with open(r'../config.yml') as file:

    parsed_yaml_file = yaml.load(file, Loader=yaml.FullLoader)

    study_area = parsed_yaml_file['study_area']
    study_area_poly_fp = parsed_yaml_file['study_area_poly']
    study_crs = parsed_yaml_file['study_crs']

    missing_tag_dict = parsed_yaml_file['missing_tag_analysis']

    incompatible_tags_dict = parsed_yaml_file['incompatible_tags_analysis']
    
print('Settings loaded!')

Settings loaded!


**Load OSM graphs and grid:**

In [150]:
# Load simplified and non-simplified graphs

osm_graph = ox.load_graphml(f'../data/osm_{study_area}.graphml')
osm_simplified_graph = ox.load_graphml(f'../data/osm_{study_area}_simple.graphml')

# Convert to nodes and edges
nodes, edges = ox.graph_to_gdfs(osm_graph)
simplified_nodes, simplified_edges = ox.graph_to_gdfs(osm_simplified_graph)

# Load grid
grid = gpd.read_file(f'../data/grid_{study_area}.gpkg')

print('Data loaded!')

Data loaded!


## Missing Tags

In [17]:
print(f'Tags to be analysed are:')
[print(value) for _, value in missing_tag_dict.items()]

missing_tags = ef.analyse_missing_tags(edges, missing_tag_dict)

for key, value in missing_tags.items():
    print(f'In the entire dataset, {value} edges or {value/len(edges)*100:.2f} % are missing information about {key}')

# TODO: Compute on a cell basis!


Tags to be analysed are:
{'true_geometries': ['surface,', 'cycleway_surface'], 'centerline': ['cycleway_surface']}
{'true_geometries': ['width,', 'cycleway_width,', 'cycleway_left_width', "cycleway_right_width'", 'cycleway_both_width'], 'centerline': ['cycleway_width', 'cycleway_left_width', 'cycleway_right_width', 'cycleway_both_width']}
{'all': ['lit']}
In the entire dataset, 46830 edges or 96.14 % are missing information about surface
In the entire dataset, 47813 edges or 98.16 % are missing information about width
In the entire dataset, 12106 edges or 24.85 % are missing information about lit


## Simplification Outcome

In [171]:
# TODO

## Network Topology

### Network Metrics
- Alpha
- Beta
- Gamma
- Number of connected components

### Signs of topological errors

"roads which do not share a common Node with another one and and lie within a radius of one meter"

"intersection but do not share a common node and not tagged bridge/tunnel"



In [ ]:
# Intersection problems

# Find all edges intersecting each other
# check if either 

def check_intersection(row, gdf):
    
    intersection = gdf[gdf.crosses(row.geometry)]

    if len(intersection) > 0 and pd.isnull(row.bridge) == True and pd.isnull(row.tunnel) == True:

        count = None

        for _, r in intersection.iterrows():

            if pd.isnull(r.bridge) == True and pd.isnull(r.tunnel) == True:
                
                print('Found problem!')
    
                count += 1

        if count > 0:
            return count

edges['intersection_issues'] = edges.apply(lambda x: check_intersection(row = x, gdf=edges), axis=1)

edges.loc[(edges.intersection_issues.notna()) & edges.intersection_issues > 0]


## Consistency

**Checking for incompatible tags**

In [59]:

incompatible_tags = ef.check_incompatible_tags(edges, incompatible_tags_dict)

print(f'In the entire dataset there are {sum(incompatible_tags.values())} incompatible tag combinations (of those defined)')

# TODO: Compute on a cell basis!


In the entire dataset there are 8 incompatible tag combinations
